<a href="https://colab.research.google.com/github/Namtk214/AIO-Homework/blob/main/aspect_base_term_extraction_BERT_%26_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets==3.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [2]:
from datasets import load_dataset
ds = load_dataset("thainq107/abte-restaurants")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/61.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3602 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [3]:
ds["train"][0]

{'Tokens': ['But', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us', '.'],
 'Tags': ['0', '0', '1', '0', '0', '0', '0', '0', '0'],
 'Polarities': ['-1', '-1', '0', '-1', '-1', '-1', '-1', '-1', '-1']}

In [4]:
corpus = [" ".join(i) for i in ds["train"]["Tokens"]]
corpus[0]

'But the staff was so horrible to us .'

In [5]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
from tokenizers.processors import TemplateProcessing as processor

tokenizer = Tokenizer(models.WordLevel(unk_token="[unk]"))

tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
trainer = trainers.WordLevelTrainer(vocab_size=5000, special_tokens=["[unk]", "[pad]"])
tokenizer.train_from_iterator(corpus, trainer=trainer)

tokenizer.save("tokenizer.json")

In [6]:
encoded = tokenizer.encode("Nam is handsome")
token_ids = [tokenizer.token_to_id(token) for token in encoded.tokens]
encoded.tokens, token_ids


(['[unk]', 'is', '[unk]'], [0, 9, 0])

In [7]:
len(tokenizer.get_vocab())

4286

In [8]:
# max len padding
MAX_LEN = max([len(token) for token in ds['train']['Tags']])
MAX_LEN

79

In [9]:

import torch

def pad_and_truncate(inputs, pad_id):
    if len(inputs) > MAX_LEN:
        inputs = inputs[:MAX_LEN]
    else:
        inputs += [pad_id] * (MAX_LEN - len(inputs))
    return inputs

def tokenize_and_align_labels(examples):
    tokenized_inputs = [] # Initialize an empty list to store tokenized inputs
    labels = []
    for tokens, tags in zip(examples["Tokens"], examples["Tags"]):
        token_ids = [tokenizer.token_to_id(token.lower()) if tokenizer.token_to_id(token.lower()) else 0
                     for token in tokens]
        tags = [int(tag) for tag in tags]
        assert len(token_ids) == len(tags)
        token_ids = pad_and_truncate(token_ids, tokenizer.token_to_id("[pad]"))

        # Remove this redundant padding:
        token_ids = pad_and_truncate(token_ids, tokenizer.token_to_id("[pad]"))
        tags = pad_and_truncate(tags, -100)

        # Append the token_ids to the tokenized_inputs list
        tokenized_inputs.append(token_ids)
        labels.append(tags)

    return {
        "input_ids": torch.tensor(tokenized_inputs),
        "labels": torch.tensor(labels)
    }


In [10]:
preprocessed_ds = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [11]:
preprocessed_ds['train']

Dataset({
    features: ['Tokens', 'Tags', 'Polarities', 'input_ids', 'labels'],
    num_rows: 3602
})

In [12]:
import torch.nn as nn
from transformers import PreTrainedModel, BertModel

class ABTELSTMClassifier(PreTrainedModel):
    def __init__(self, config, vocab_size, num_classes, embedding_dim=256, hidden_dim=256, pad_idx=0):
        super().__init__(config)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, labels=None):
        embeddings = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embeddings)
        logits = self.fc(lstm_out)

        loss = None
        if labels is not None:
            loss = self.loss(logits.permute(0, 2, 1), labels)
        return {"loss": loss, "logits": logits}

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [13]:
from transformers import PretrainedConfig
config = PretrainedConfig()
model = ABTELSTMClassifier(config, vocab_size=len(tokenizer.get_vocab()), num_classes=3)

In [14]:
model

ABTELSTMClassifier(
  (embedding): Embedding(4286, 256, padding_idx=0)
  (lstm): LSTM(256, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=3, bias=True)
  (loss): CrossEntropyLoss()
)

In [15]:
# evaluate
!pip install -q seqeval==1.2.2
import numpy as np


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [16]:
id2label = {0: "O", 1: "B-RES", 2: "I-RES"}
label2id = {"O": 0, "B-RES": 1, "I-RES": 2}

In [17]:
from seqeval.metrics import f1_score, classification_report
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = classification_report(true_labels, true_predictions)
    return {"f1": f1_score(true_labels, true_predictions)}

In [25]:
# training
from transformers import TrainingArguments, Trainer
import os
#os.environ["WANDB_DISABLED"] = "true"
training_args = TrainingArguments(
    output_dir="abte-restaurant-LSTM",
    logging_dir="logs",
    learning_rate=2e-5,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs = 10,
    weight_decay = 0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["test"],
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.719700,0.725432,0.054088
2,0.687400,0.705223,0.054020
3,0.663400,0.690491,0.054470
4,0.641100,0.680827,0.048906
5,0.633400,0.675065,0.048875
6,0.621700,0.671384,0.044516
7,0.612600,0.669056,0.049550
8,0.608900,0.667580,0.049597
9,0.610000,0.666662,0.049597
10,0.604500,0.666303,0.049597


TrainOutput(global_step=150, training_loss=0.6402872975667318, metrics={'train_runtime': 175.2694, 'train_samples_per_second': 205.512, 'train_steps_per_second': 0.856, 'total_flos': 8999550822360.0, 'train_loss': 0.6402872975667318, 'epoch': 10.0})

BERT PRETRAINED MODEL

In [29]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [44]:
from transformers import AutoTokenizer

def tokenize_and_align_labels(examples):
    tokenized_inputs = []
    labels = []
    for tokens, tags in zip(examples["Tokens"], examples["Tags"]):
        bert_tokens = []
        bert_tags = []
        for i in range(len(tokens)):
            t = tokenizer.tokenize(tokens[i])
            bert_tokens += t
            bert_tags += [int(tags[i])]*len(t)
        bert_ids = tokenizer.convert_tokens_to_ids(bert_tokens)
        tokenized_inputs.append(bert_ids)
        labels.append(bert_tags)
    return {
        "input_ids": (tokenized_inputs),
        "labels": (labels)
    }

preprocessed_ds = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [45]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels =3, id2label=id2label , label2id =id2label)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
# training
from transformers import TrainingArguments, Trainer
import os
#os.environ["WANDB_DISABLED"] = "true"
training_args = TrainingArguments(
    output_dir="abte-restaurant-BERT-PRETRAINED",
    logging_dir="logs",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs = 5,
    weight_decay = 0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["test"],
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [49]:
trainer.train()

ValueError: expected sequence of length 28 at dim 1 (got 7)